In [1]:
with open("input.txt") as f:
    lines = [
        line.strip() for line in f.readlines() if line.strip()
    ]

In [2]:
def rotate_clockwise(grid):
    """Rotates a 2D list clockwise"""
    return list(list(row) for row in zip(*grid[::-1]))

def rotate_string_grid_clockwise(grid):
    """Rotates a list of strings clockwise"""
    return ["".join(line) for line in rotate_clockwise(grid)]

def flip_horizontal(grid):
    """Flipes a 2D list along the vertical axis"""
    return [line[::-1] for line in grid]

def flip_vertical(grid):
    """Flipes a 2D list along the horizontal axis"""
    return list(reversed(grid))

In [3]:
from typing import List, Set

class Tile:

    def __init__(self, tile_id: int, grid: List[List[str]]):
        self.tile_id = tile_id
        self.grid = grid
        self._compute_lrtb()
        self.all_sides = {self.top, self.bottom, self.left, self.right}
        self.flip_horizontal()
        self.all_sides |= {self.top, self.bottom}
        self.flip_vertical()
        self.all_sides |= {self.left, self.right}
        self.flip_horizontal()
        self.flip_vertical()

    def _compute_lrtb(self):
        """Sets the left, right, top, bottom based off of self.grid"""
        self.top = "".join(self.grid[0])
        self.bottom = "".join(self.grid[-1])
        self.left = "".join([line[0] for line in self.grid])
        self.right = "".join([line[-1] for line in self.grid])

    def rotate_clockwise(self):
        """Rotates the tile clockwise"""
        self.grid = rotate_string_grid_clockwise(self.grid)
        self._compute_lrtb()

    def flip_horizontal(self):
        """Flips the tile along the vertical axis (i.e. left to right)"""
        self.grid = flip_horizontal(self.grid)
        self._compute_lrtb()

    def flip_vertical(self):
        """Flips along the horizontal axis"""
        self.grid = flip_vertical(self.grid)
        self._compute_lrtb()

    def any_matches(self, other_tile) -> bool:
        """Returns whether any configuration allows two tiles to line up"""
        return bool(
            self.all_sides.intersection(other_tile.all_sides)
        )

    def __eq__(self, other_tile):
        return self.tile_id == other_tile.tile_id

    def __repr__(self):
        return str(self.tile_id) + "\n" + "\n".join(self.grid)

Start by reading the input and mapping the tile id's to `Tile`
objects.

In [4]:
from collections import defaultdict
import re

id_to_tile = defaultdict(list)
last_tile = None

for line in lines:

    if match := re.fullmatch("Tile (\d+):", line):
        last_tile = int(match.group(1))
    else:
        id_to_tile[last_tile].append(line)

id_to_tile = {
    tile_id: Tile(tile_id, grid) for tile_id, grid in id_to_tile.items()
}

## part1

We take a bit of a shortcut here. For each tile, we find all other tiles
that could possibly be a neighbor (there needs to be a side shared between
the two tiles). If a tile only has 2 possible neighbors, then it must be a
corner tile.

In [5]:
from functools import reduce
from operator import mul

tile_id_to_neighbors = {
    tile_id: set(
        other_tile.tile_id
        for other_tile in id_to_tile.values()
        if tile.any_matches(other_tile) and tile != other_tile
    )
    for tile_id, tile in id_to_tile.items()
}

corner_tiles = set(
    tile_id
    for tile_id, neighbors in tile_id_to_neighbors.items()
    if len(neighbors) == 2
)
side_tiles = set(
    tile_id
    for tile_id, neighbors in tile_id_to_neighbors.items()
    if len(neighbors) == 3
)
interior_tiles = set(
    tile_id
    for tile_id, neighbors in tile_id_to_neighbors.items()
    if len(neighbors) == 4
)

reduce(mul, (tile_id for tile_id in corner_tiles))

19955159604613

## part2

Now we need to completely fill out the grid with all of the
tiles oriented correctly. We do this in two steps

1. Fill the grid out with which tile goes in which spot
2. Go back over the grid and flip/rotate each tile until it's
in the correct orientation

To start, we pick a corner tile and place it the `(0, 0)` position
and place its two neighbors in the `(0, 1)` and `(1, 0)` position.

In [6]:
# we have 144 total tiles making a 12x12 square grid
DIMENSIONS = 12

grid = [[None] * DIMENSIONS for _ in range(DIMENSIONS)]

available_tile_ids = set(
    tile_id for tile_id in corner_tiles | side_tiles | interior_tiles
)

upper_left_tile = list(corner_tiles)[0]
grid[0][0] = upper_left_tile
grid[0][1], grid[1][0] = tuple(tile_id_to_neighbors[upper_left_tile])

available_tile_ids.remove(upper_left_tile)
available_tile_ids.remove(grid[0][1])
available_tile_ids.remove(grid[1][0])

No we fill out all of the tiles in the grid. We have three cases

1. Corner tiles (two neighbors)
2. Sides tiles (three neighbors)
3. Interior tiles (four neighbors)

We fill out the grid row-by-row so for each tile, we should have
already filled out the tile above and to the left of it (given
that it's not on the top row or left column).

We rely on the `tile_id_to_neighbors` dictionary which maps a tile
ID to its set of neighbors (tiles that share sides).

When filling out a corner tile, we look look at a neighboring tile.
The neighboring tile should only have one possible neighbor which
is also a corner tile so that gives us the tile that we're looking
for.

When filling out a side tile, we look at a neighboring tile. The
neighboring tile should only have one possible neighbor which is
a side tile and has not already been placed in the grid so that
gives us the tile that we're looking for.

Finally, when filling out an interior tile we look at two neighbors
and find the intersection of their neighboring tiles along with the
intersection of interior tiles and tiles that have not already been
placed in the grid. The unique intersection of those gives us the
tile that we're looking for.

In [7]:
CORNERS = {
    (0, 0), (0, DIMENSIONS-1), (DIMENSIONS-1, 0), (DIMENSIONS-1, DIMENSIONS-1)
}
SIDES = {0, DIMENSIONS-1}

def is_corner(row, col):
    return (row, col) in CORNERS

def is_side(row, col):
    return row in SIDES or col in SIDES and not is_corner(row, col)

def is_interior(row, col):
    return not is_corner(row, col) and not is_side(row, col)

for row in range(DIMENSIONS):
    for col in range(DIMENSIONS):

        if grid[row][col] is not None:
            continue

        if is_corner(row, col):
            if col == 0:
                neighbor = grid[row-1][col]
            else:
                neighbor = grid[row][col-1]

            candidate_tiles = corner_tiles & tile_id_to_neighbors[neighbor] & available_tile_ids

        elif is_side(row, col):
            if row == 0 or row == DIMENSIONS - 1:  # top or bottom row
                neighbor = grid[row][col-1]
            elif col == 0 or col == DIMENSIONS - 1:  # left or right side
                neighbor = grid[row-1][col]

            candidate_tiles = side_tiles & tile_id_to_neighbors[neighbor] & available_tile_ids

        else:  # interior tile
            # check tiles above and to the left, there should only be one that
            # matches both
            tile_above = grid[row-1][col]
            tile_left = grid[row][col-1]
            candidate_tiles = interior_tiles & available_tile_ids
            candidate_tiles &= tile_id_to_neighbors[tile_above]
            candidate_tiles &= tile_id_to_neighbors[tile_left]

        assert len(candidate_tiles) == 1
        this_tile = candidate_tiles.pop()
        available_tile_ids.remove(this_tile)
        grid[row][col] = this_tile

Now that the grid is filled out with tile IDs, we create a new grid
to hold the corresponding `Tile` object rotated and flipped to the
correct orientation.

For each tile in the grid, we start by finding its left, right, up,
and down neighbors (a tile will have fewer than 4 neighbors if it's
on an edge). Then, we flip and rotate the tile in all orientations
until its left side is shared with the tile to its left, its top is
shared with the tile above, and so on.

Once this is complete, all tiles are oriented correctly in the grid.

In [8]:
tile_grid = [[None] * DIMENSIONS for _ in range(DIMENSIONS)]

for row in range(DIMENSIONS):
    for col in range(DIMENSIONS):
        this_tile = id_to_tile[grid[row][col]]
        tile_left = tile_right = tile_above = tile_below = None
        if col > 0:
            tile_left = id_to_tile[grid[row][col-1]]
        if col < DIMENSIONS - 1:
            tile_right = id_to_tile[grid[row][col+1]]
        if row > 0:
            tile_above = id_to_tile[grid[row-1][col]]
        if row < DIMENSIONS - 1:
            tile_below = id_to_tile[grid[row+1][col]]

        orientation_found = False
        horizontal_flip = False

        while not orientation_found:

            if horizontal_flip:
                this_tile.flip_horizontal()
            else:
                this_tile.flip_vertical()

            horizontal_flip = not horizontal_flip

            for _ in range(4):
                left_found = tile_left is None or this_tile.left in tile_left.all_sides
                right_found = tile_right is None or this_tile.right in tile_right.all_sides
                top_found = tile_above is None or this_tile.top in tile_above.all_sides
                bottom_found = tile_below is None or this_tile.bottom in tile_below.all_sides

                orientation_found = left_found and right_found and top_found and bottom_found
                if orientation_found:
                    break

                this_tile.rotate_clockwise()

        tile_grid[row][col] = this_tile

Finally, we generate the grid of strings witb all of the borders
removed. We start by copying the grid into the `string_grid` array
and chopping off the borders.

Then, we go row-by-row and concatenate each individual row together
into one line so that we end up with a 1D list of strings stored in
`string_grid`.

In [9]:
string_grid = [[None] * DIMENSIONS for _ in range(DIMENSIONS)]

for row in range(DIMENSIONS):
    for col in range(DIMENSIONS):
        this_tile = tile_grid[row][col]
        string_grid[row][col] = [line[1:-1] for line in this_tile.grid[1:-1]]

string_grid = [
    "".join([grid[row] for grid in tile_row])
    for tile_row in string_grid
    for row in range(len(tile_row[0]))
]

Our very last step, we have to count the number of `#` that
aren't part of a sea monster. We start by counting the number
of `#` in `string_grid`. Then, we rotate and flip the grid
into all orientations search row-by-row for sea monsters using
regular expressions. When we find one, we mark all `#` part of
the sea monster in the 2D list `hashes_not_in_monster` as `False`.

In [10]:
# These three patterns form the "monster" when found # on
# 20 characters vertically aligned on three adjacent rows.
first_row_regex = ".{18}#."
second_row_regex = "#.{4}##.{4}##.{4}###"
third_row_regex = ".#..#..#..#..#..#..."

hashes_not_in_monster = [
    [char == "#" for char in line]
    for line in string_grid
]

# We need to do all combinations of flipping and rotating the
# grid. We start with the grid as-is, rotate it clockwise 360
# degrees, then flip it horizontally, and repeat (flipping it
# vertically once and then horizontally again).
for flip_func in (flip_horizontal, flip_vertical, flip_horizontal):
    for _ in range(4):
        for row_num in range(len(string_grid) - 2):
            # Grab three adjacent rows
            first_row = string_grid[row_num]
            second_row = string_grid[row_num+1]
            third_row = string_grid[row_num+2]

            for start_index in range(len(first_row)-20):
                # grab 20 characters that are vertically aligned from each row
                first_str = first_row[start_index:start_index+20]
                second_str = second_row[start_index:start_index+20]
                third_str = third_row[start_index:start_index+20]

                first_row_match = re.fullmatch(first_row_regex, first_str)
                second_row_match = re.fullmatch(second_row_regex, second_str)
                third_row_match = re.fullmatch(third_row_regex, third_str)

                # If each row matches the monster pattern, we've found the
                # monster pattern and need to mark each # part of the
                # monster as False in the hashes_not_in_moster grid.
                if first_row_match and second_row_match and third_row_match:
                    hashes_not_in_monster[row_num][start_index + 18] = False

                    # These are the indices of the # forming the monster in the
                    # pattern. They are offset from the start index of the 20
                    # characters that we're currently looking at.
                    for offset in (0, 5, 6, 11, 12, 17, 18, 19):
                        hashes_not_in_monster[row_num+1][start_index + offset] = False

                    for offset in (1, 4, 7, 10, 13, 16):
                        hashes_not_in_monster[row_num+2][start_index + offset] = False

        string_grid = rotate_string_grid_clockwise(string_grid)
        hashes_not_in_monster = rotate_clockwise(hashes_not_in_monster)

    string_grid = flip_func(string_grid)
    hashes_not_in_monster = flip_func(hashes_not_in_monster)

Count the number of `#` not part of a sea monster, and we're done!

In [11]:
sum(
    is_hash_char
    for line in hashes_not_in_monster
    for is_hash_char in line
)

1639